In [ ]:
import os
import torch
from vggt.models.vggt import VGGT

from vggt.utils.load_fn import load_and_preprocess_images

device = "cuda" if torch.cuda.is_available() else "cpu"
# bfloat16 is supported on Ampere GPUs (Compute Capability 8.0+) 
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

# Initialize the model and load the pretrained weights.
# This will automatically download the model weights the first time it's run, which may take a while.
model = VGGT.from_pretrained("facebook/VGGT-1B").to(device)




['people/images/WechatIMG199.jpg', 'people/images/WechatIMG200.jpg', 'people/images/WechatIMG201.jpg', 'people/images/WechatIMG202.jpg', 'people/images/WechatIMG203.jpg']


In [12]:
# Read images from 'people/images' directory
# image_dir = 'examples/room/images'
image_dir = 'people/images'
image_names = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
print(image_names)

['people/images/WechatIMG199.jpg', 'people/images/WechatIMG200.jpg', 'people/images/WechatIMG201.jpg', 'people/images/WechatIMG202.jpg', 'people/images/WechatIMG203.jpg']


In [16]:
# reorder the fourth image to be the first
index_to_move = 0
image_names.insert(4, image_names.pop(index_to_move))
print(image_names)


['people/images/WechatIMG200.jpg', 'people/images/WechatIMG201.jpg', 'people/images/WechatIMG202.jpg', 'people/images/WechatIMG203.jpg', 'people/images/WechatIMG199.jpg']


In [13]:
from vggt.utils.pose_enc import pose_encoding_to_extri_intri
from vggt.utils.geometry import unproject_depth_map_to_point_map

images = load_and_preprocess_images(image_names, mode='pad').to(device)

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=dtype):
        images = images[None]  # add batch dimension
        aggregated_tokens_list, ps_idx = model.aggregator(images)
                
    # Predict Cameras
    pose_enc = model.camera_head(aggregated_tokens_list)[-1]
    # Extrinsic and intrinsic matrices, following OpenCV convention (camera from world)
    extrinsic, intrinsic = pose_encoding_to_extri_intri(pose_enc, images.shape[-2:])

    # Predict Depth Maps
    depth_map, depth_conf = model.depth_head(aggregated_tokens_list, images, ps_idx)

    # Predict Point Maps
    point_map, point_conf = model.point_head(aggregated_tokens_list, images, ps_idx)
        
    # Construct 3D Points from Depth Maps and Cameras
    # which usually leads to more accurate 3D points than point map branch
    point_map_by_unprojection = unproject_depth_map_to_point_map(depth_map.squeeze(0), 
                                                                extrinsic.squeeze(0), 
                                                                intrinsic.squeeze(0))



In [14]:
with torch.no_grad():
    # Predict Tracks
    # choose your own points to track, with shape (N, 2) for one scene
    query_points = torch.FloatTensor([[224.4112, 187.6958],
          [251.5651, 189.3672]]).to(device)
    track_list, vis_score, conf_score = model.track_head(aggregated_tokens_list, images, ps_idx, query_points=query_points[None])

In [15]:
from vggt.utils.visual_track import visualize_tracks_on_images
track = track_list[-1]
visualize_tracks_on_images(images, track, (conf_score>0.2) & (vis_score>0.2), out_dir="track_visuals")

[INFO] Saved color-by-XY track visualization grid -> track_visuals/tracks_grid.png
[INFO] Saved 5 individual frames to track_visuals/frame_*.png


In [10]:
track

tensor([[[[172.0000, 172.0000],
          [340.0000, 180.9400]],

         [[224.4112, 187.6958],
          [251.5651, 189.3672]],

         [[282.4581, 151.3021],
          [244.2977, 155.3082]],

         [[221.4864, 449.5332],
          [269.3793, 388.7892]],

         [[413.5035, 145.3219],
          [440.6994, 243.9181]]]], device='cuda:0')